In [11]:
import pandas as pd
import numpy as np
import os
import sqlite3
import shutil
import datetime
import matplotlib.pyplot as plt
import tabulate
from IPython.display import HTML, display, Markdown
import csv
import unicodedata
import getopt
from ipywidgets import HBox, VBox, Layout
from matplotlib.ticker import MaxNLocator

import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout
import graphviz
from GraphVizUtil import *
from GraphVizFormats import *
import warnings
warnings.filterwarnings('ignore')
import seaborn as sb
sb.set(style='darkgrid', font_scale=1.2)

def filter_descriptions(tech_comm_desc):
    try:
        tech_comm_desc = tech_comm_desc.values[0][0].replace('#', '').replace('"','').replace("\n",'').strip()
    except:
        tech_comm_desc = 'No description provided'
    return tech_comm_desc

def create_args_flowd(df_graph):
    nodes, tech, ltech, to_tech, from_tech = set(), set(), set(), set(), set()
    for ind,row in df_graph.iterrows():
        #descriptions:
        input_comm_des = filter_descriptions(pd.read_sql("SELECT comm_desc FROM commodities WHERE comm_name='" + row['input_comm'] + "'", con))
        output_comm_des = filter_descriptions(pd.read_sql("SELECT comm_desc FROM commodities WHERE comm_name='" + row['output_comm'] + "'", con))
        tech_des = filter_descriptions(pd.read_sql("SELECT tech_desc FROM technologies WHERE tech='" + row['tech'] + "'", con))

        if 'ethos' in row['input_comm']:
            ltech.add('"' + row['tech'] + '"' +  ' [tooltip = "' + tech_des + '"]')
        else :
            nodes.add('"' + row['input_comm'] + '"' +  ' [tooltip = "' + input_comm_des + '"]')
        nodes.add('"' + row['output_comm'] + '"' +  ' [tooltip = "' + output_comm_des + '"]')
        tech.add('"' + row['tech'] + '"' +  ' [tooltip = "' + tech_des + '"]')

        if row['input_comm'] != 'ethos':
            to_tech.add('"%s"' % row['input_comm'] + '\t->\t"%s"' % row['tech']) 
        from_tech.add('"%s"' % row['tech'] + '\t->\t"%s"' % row['output_comm'])
    args = dict(
    enodes = "".join('%s;\n\t\t' % x for x in nodes),
    tnodes = "".join('%s;\n\t\t' % x for x in tech),
    iedges = "".join('%s;\n\t\t' % x for x in to_tech),
    oedges = "".join('%s;\n\t\t' % x for x in from_tech),
    snodes = ";".join('%s' %x for x in ltech),
    )
    return args


def return_format_colors():
    colors = {}
    colors.update(getColorConfig(False))
    return colors, quick_run_dot_fmt

def return_flowd_table(final_dem, level=1):
    df = pd.read_sql("SELECT * FROM Efficiency", con)
    df_sel = df[df['output_comm']==final_dem]
    if len(df_sel)==0:
        df_sel = df[df['tech']==final_dem]
    inputs = df_sel['input_comm'].unique()
    iterval=0
    if level!=0:
        while len(inputs)>0:
            df_append = df[df['output_comm'].isin(inputs)]
            df_sel = pd.concat([df_sel, df_append])
            inputs = df_append['input_comm'].unique()
            iterval+=1
            if iterval>level-1:
                break
    df_graph = df_sel[['input_comm', 'tech', 'output_comm']].drop_duplicates()
    return df_graph

def return_flowd_table_fwds(final_dem):
    df = pd.read_sql("SELECT * FROM Efficiency", con)
    df_sel = df[df['output_comm']==final_dem]
    if len(df_sel)==0:
        df_sel = df[df['tech']==final_dem]
    inputs = df_sel['input_comm'].unique()
    outputs = df_sel['output_comm'].unique()

    iterval=0
    while len(inputs)>0:
        df_append = df[df['output_comm'].isin(inputs)]
        df_sel = pd.concat([df_sel, df_append])
        inputs = df_append['input_comm'].unique()
        iterval+=1
        if iterval>2:
            break
    iterval=0
    while len(outputs)>0:
        df_append = df[df['input_comm'].isin(outputs)]
        df_sel = pd.concat([df_sel, df_append])
        outputs = df_append['output_comm'].unique()
        iterval+=1
        if iterval>=0:
            break

    df_graph = df_sel[['input_comm', 'tech', 'output_comm']].drop_duplicates()
    return df_graph

con = sqlite3.connect(r'../US_9R_4D.sqlite') #change path to database
cur = con.cursor()   
con.text_factory = str 

def controls_rows(w):
    controls = HBox(w.children[:-1], layout = Layout(flex_flow='row wrap', width='max-content'))
    output = w.children[-1]
    display(VBox([controls, output],  layout = Layout(flex_flow='columns wrap', width='max-content', size=10)))
    

# Buildings Sector Overview <a class="anchor" id="buildings"></a>
The buildings sector in the OEO database is subdivided into the residential and commercial sectors. These sectors consist of two key components: end-use service demands and the demand technologies used to meet those demands. The service demands in turn consist of two components: the region-specific annual demands projected to 2050, and the apportionment of these annual demands at the hourly level. The demand technologies are characterized by their capital costs, fixed and variable operation and maintenance costs, efficiencies, capacity factors, existing capacity, and fuel inputs.

## Service Demands <a class="anchor" id="service_demands"></a>
The <a href="https://www.nrel.gov/analysis/electrification-futures.html" rel="nofollow">NREL Electrification Futures Study</a> used the EnergyPATHWAYS tool to develop a set of energy demand and supply projections for
the end-use sectors (including buildings) to 2050. These annual projections are available at the state level for subsectors such as space heating, air conditioning, cooking, and water heating. The study also has state-level
projections of hourly electricity demands for some of these subsectors. We are currently using these two datasets to develop demand projections for the OEO database. In the future, we plan to utilize more detailed projections based on <a href="https://www.nrel.gov/buildings/end-use-load-profiles.html" rel="nofollow">this multi-year NREL study</a> that is currently in progress.


### Annual Demand Projections <a class="anchor" id="annual_demands"></a>
The <a href="https://www.nrel.gov/docs/fy18osti/71500.pdf" rel="nofollow">NREL Electrification Futures Study (EFS) report</a> was published in 2018 and presents a range of demand scenarios for all energy sectors in the United States. Specifically, the study estimates building-related service demands, as shown in Table 1, primarily by downscaling service demands data from the US EIA 2017 Annual Energy Outlook and the associated NEMS model.

EIA service demand data were made available by US Census Division to the EFS study. These Census Division-level data were then downscaled to the state-level in the EFS by using factors relevant to each end-use demand. For example, residential space heating service demands at the Census Division were downscaled to the state-level by taking the product of each state's share of heating-degree-days and residential square footage within a given region. Similarly, residential water heating was downscaled by using the number of households. These state-level demand projections to 2050 were made available in the [EFS](https://data.nrel.gov/submissions/92) for the demand types listed in Table 1.

We aggregated these state-level demands by demand category, to the nine OEO regions shown in Figure 1.

**Table 1.** Annual service demand categories and units drawn from the EFS and used in the OEO buildings sector. The demand commodity names used in the OEO input database are also presented.

| End-Use Demand | Unit | OEO Demand Commodity |
|:-|:-|:-|
| commercial space cooling <img width=15/> | TERABTU |CSC<img width=15/>|
| commercial cooking | TERABTU | CCK |
| commercial lighting | GIGALUMEN_YEAR |CLT |
| commercial   refrigeration | TERABTU | CRF |
| commercial space   heating | TERABTU | CSH |
| commercial   ventilation | GIGACUBIC_FOOT | CVT |
| commercial water   heating | TERABTU | CWH |
| residential   space cooling | MMBTU | RSC |
| residential refrigeration | CUBIC_FEET | RRF |
| residential   lighting | KILOLUMEN_HOUR | RLT |
| residential   space heating | MMBTU | RSH |
| residential   water heating | MMBTU | RWH |

###  Hourly Demand Representation <a class="anchor" id="hourly_demands"></a>

As part of the EFS, NREL has developed and published [hourly level electricity demand profiles](https://data.nrel.gov/submissions/126) (for 8760 hours a year) for several demand categories listed in Table 1. NREL has used the outputs of models such as ResStock and ComStock and other data (e.g., from metering studies) to develop these representative hourly profiles for a range of future scenarios. Although these profiles have been developed for electricity demands only, we assume that the service demands follow the same hourly demand pattern. We use the electrification profiles from the 'High Electrification, Rapid End-use Technology Advancement' scenario for our estimates (service demands listed in Table 1), since that scenario incorporates the largest share of total service demands. For the subsectors for which no hourly profiles are available, we assume service demands are constant throughout the year. 

![Figure 1: IPM regions](./documentation_images/states_to_temoa_state_groups.jpg)
**Figure 1.** The nine regions represented in the Open Energy Outlook. The grey lines show the Integrated Planning Model (IPM) region boundaries.

The NREL EFS report combines space heating and cooling load profiles. To separate these profiles, we use population weighted heating and cooling degree hour (hdh/cdh) data provided by Michael Roberts at the University of Hawaii, which is described in a forthcoming paper. This data is available at the hourly timescale for the year 2010 and is available across the IPM regions. We first calculate an average hdh for IPM regions within a given OEO region (Figure 1) for each hour of 2010. Next, we calculate the hourly hdh fraction as: 
\begin{equation*}
\frac{\bar{hdh}}{\bar{hdh} + \bar{cdh}}
\end{equation*}
where $\bar{hdh}$ and $\bar{cdh}$ are the hourly hdh and cdh, respectively, averaged across IPM regions in a given OEO region. These fractions are calculated for each hour in each OEO region. Finally, the combined heating and cooling load from NREL EFS is multiplied by these fractions to estimate the heating load in each hour. The cooling load is similarly estimated as the total load multiplied by the fraction of cooling degree hours over the sum of heating and cooling degree hours. This disaggregation of hourly heating and cooling load is illustrated in Figure 2 for two OEO regions.

![Figure 2: hdh and hdh fraction for OEO regions](./documentation_images/hdh_cdh_OEO.png)
**Figure 2.** Heating degree hour (hdh) and fraction of heating degree hour for the OEO ‘TX’ (left) and 'CEN' (right) region. The markers on the plot show the hourly hdh for the IPM regions within the given OEO region. The line shows the average hdh fraction for each hour smoothed with a moving average window of 20 hours for legibility.

In [12]:
display(Markdown('The hourly demand profiles for residential/commercial end use demand are shown below. Refer to Table 2 for a description of the technologies'))

def show_hourly_dem(conn):
    df_dds = pd.read_sql_query('SELECT * FROM DemandSpecificDistribution', conn)
    regions = df_dds.regions.unique()
    unique_demands = df_dds.demand_name.unique()
    
    def select_region(region='', dem_name=''):
        df_sel = df_dds.loc[(df_dds.regions==region) & (df_dds.demand_name==dem_name)].copy()
        fig, ax = plt.subplots(figsize=(10,6))
        plt.plot(np.arange(0, len(df_sel['dds'])),df_sel['dds'], label=dem_name)
        plt.ylabel('Hourly demand fraction')
        plt.xlabel('Hour')
        plt.legend(loc='upper right',bbox_to_anchor=(1.2, 1))

    w1 = widgets.Select(options=regions)
    w2 = widgets.Select(options=unique_demands)
    w = widgets.interactive(select_region, region=w1, dem_name=w2)
    
    controls_rows(w)

show_hourly_dem(con)

The hourly demand profiles for residential/commercial end use demand are shown below. Refer to Table 2 for a description of the technologies

In [13]:
display(Markdown('**Table 2.** Demand categories that have hourly resolution in the OEO database'))
df = pd.read_sql("SELECT comm_name, comm_desc FROM commodities \
	WHERE comm_name IN \
	(SELECT DISTINCT(demand_name) FROM DemandSpecificDistribution)", con)
df.columns=['Demand','Description']
df['Description'] = df['Description'].str.replace('#','').str.strip()
display(HTML(tabulate.tabulate(df.set_index('Demand'),df.columns,tablefmt='html')))

**Table 2.** Demand categories that have hourly resolution in the OEO database

Demand,Description
CLT,commercial lighting demand
COEELC,commercial electric demand for office equipment
COELC,commercial other electric demand
CSC,commercial space cooling demand
CSH,commercial space heating demand
CWH,commercial water heating demand
RLT,residential lighting demand
ROELC,residential other electric demand
RSC,residential space cooling demand
RSH,residential space heating demand


## Demand Technology Specification <a class="anchor" id="demand_technology"></a>

The characteristics of the demand technologies within the residential and commercial sectors are based on the Residential Demand Module (RDM) and Commercial Demand Module (CDM) of the National Energy Modeling System (NEMS) - [Updated Buildings Sector Appliance and Equipment Costs and Efficiency](https://www.eia.gov/analysis/studies/buildings/equipcosts/). These modules provide technology-specific efficiencies, fixed and variable operations and maintenance costs, lifetimes, and typical capacities based on contract reports prepared by Navigant Consulting, Inc. for the U.S. Energy Information Administration. These modules include a diverse set of equipment classes and types. For example, residential cooling technologies include room air conditioners, central air conditioners, and heat pumps. Three types of heat pumps are represented for residential cooling in this database: air-source, ground-source, and natural gas heat pumps. We used this dataset as-is, except for an adjustment of the heat pump coefficient of performance (COP) to reflect more regionally appropriate values. We perform this transformation since heat pumps are an important technology pertaining to building decarbonization. [Vaishnav et al. (2020)](https://pubs.acs.org/doi/abs/10.1021/acs.est.0c02705) used a dataset gathered from NEEP and estimated a linear relationship between COP and outside temperature for a variety of heat pumps. Their approach assumed that the indoor temperatures stay constant at 70 F. We use the slope of this linear relationship to adjust heat pump COPs at the OEO region level using the NEMS database and applying state-level population weighted temperatures.
    
In addition to representing a wide array of technologies, the current and projected techno-economic parameters for the residential and commercial equipment are projected to 2050. Techno-economic parameter estimates are provided for typical equipment in a given year along with estimates for high efficiency versions of the same equipment in the same year. For example: costs and coefficients of performance (COPs) for a typical air-source heat pump as well as a high efficiency air-source heat pump are published for vintages spanning 2020 through 2050 in 5 year increments. We utilize this data as part of the OEO buildings database. In addition, techno-economic parameters for a technology - wit the exception of heat pump COPs, as described above - are assumed to be the same across regions. 


Input values drawn directly from the database can be viewed in the sections below.

### Investment costs <a class="anchor" id="costinvest"></a>

The investment costs are drawn from the [Residential Demand Module (RDM) and Commercial Demand Module (CDM)](https://www.eia.gov/analysis/studies/buildings/equipcosts/) of National Energy Modeling System.

In [14]:
def show_cost_invest(con):
    
    df_tech_desc = pd.read_sql("SELECT tech, tech_desc FROM technologies", con)

    df = pd.read_sql(
        "SELECT regions, tech,vintage, cost_invest, cost_invest_units FROM CostInvest WHERE tech IN (SELECT tech FROM technologies WHERE sector=='residential' OR sector='commercial' ) ORDER BY tech, vintage",
        con)

    display_types = ['table', 'figure']
    
    df['agg_tech'] = df['tech'].map(lambda x: df_tech_desc.loc[df_tech_desc.tech==x,'tech_desc'].values[0].replace('#','').strip())
    techs = []
    for unique_tech in df.agg_tech.unique():
        try:
            int(unique_tech.split('_')[-1])
            techs.append('_'.join(unique_tech.split('_')[0:-1]))
        except:
            techs.append(unique_tech)
    techs = ['All'] + list(set(techs))
    regions = df.regions.unique()


    def filter_tech(tech='', region='', o_format=''):
        if tech == 'All':
            df_sel = df[(df.regions == region)]
        else:
            df_sel = df[(df.agg_tech.str.contains(tech)) & (df.regions == region)]
        if o_format == 'table':
            df_sel = df_sel.pivot_table(
                index=['regions', 'tech', 'agg_tech', 'cost_invest_units'],
                columns='vintage',
                values='cost_invest').reset_index().set_index('regions')
            df_sel.rename(columns={'cost_invest_units': 'units', 'agg_tech':'description'}, inplace=True)
            df_sel['units'] = df_sel['units'].str.replace('#','').str.replace('M$','$M').str.strip()
            if len(df_sel['units'].unique())==1:
                df_sel['units'] = df_sel['units'].unique()[0].replace('M$','$M')
            display(
                HTML(
                    tabulate.tabulate(df_sel,
                                      ['region'] + list(df_sel.columns.values),
                                      floatfmt=".0f",
                                      tablefmt='html')))
        elif (o_format == 'figure') & (tech!='All') :
            fig, ax = plt.subplots(figsize=(10, 6))
            for ind_tech in df_sel.tech.unique():
                plt.plot(df_sel[df_sel.tech == ind_tech].vintage,
                         df_sel[df_sel.tech == ind_tech].cost_invest,
                         label=ind_tech)
            plt.legend()
            plt.ylabel('Investment costs (' + df_sel['cost_invest_units'].unique()[0].replace('#','').strip().replace('M$','$M') + ')')
            #plt.ylim([0, df.cost_invest.max() * 1.1])
            plt.xlabel('Vintage')


    w1 = widgets.Select(options=techs)
    w2 = widgets.Select(options=regions)
    w3 = widgets.Select(options=display_types)
    w = widgets.interactive(filter_tech, tech=w1, region=w2, o_format=w3)

    controls_rows(w)
    
show_cost_invest(con)

### Efficiency <a class="anchor" id="efficiency"></a>

The efficiency parameter has two functions in the OEO database: (1) specify the conversion efficiency of each process, and (2) specify linkages in input and output processes to build the energy system network. 

As shown in Table 1, our representation of the buildings sector is comprised of several end-use services like residential space heating and commercial ventilation. These end-use service demands are met by a host of demand technologies that consume energy commodities and produce end-use demand commodities. As shown in Table 2, the numerous efficiency metrics reflect the wide range of demand technologies included in to OEO buildings representation.

The demands for some end-use services (like residential space heating) are specified in energy units and so the efficiencies of technologies meeting these demands are in units of energy output over energy input supplied. However, other end-use services (like commercial ventilation) have demand specified in physical units, and thus the efficiencies of the associated demand technologies are in physical units produced over input energy supplied. Table 2 below shows the efficiency metrics that apply to different categories of demand technologies serving a specific end-use demand.

**Table 2.** Summary of technology efficiencies applied to building subsectors in the OEO database. 

| End-Use Demand | Efficiency Metrics | Efficiency Units
|:-|:-|:-|
| Commercial Space Cooling <img width=15/> | COP<sup>[1](#COP)</sup>, EER<sup>[2](#EER)</sup>, IEER<sup>[2](#EER)</sup> <img width=15/>| PJ out/PJ In<img width=15/>|
| Commercial Cooking | cooking energy efficiency | PJ out/PJ In |
| Commercial Lighting | system efficacy<sup>[3](#LIGHT)</sup> | Lumen-Year/PJ |
| Commercial Refrigeration | Nominal Capacity Over Average Input | PJ out/PJ in |
| Commercial Space Heating | COP, thermal efficiency<sup>[4](#TE)</sup>, COP, AFUE<sup>[5](#AFUE)</sup>| PJ out/PJ in |
| Commercial Ventilation | system air flow over fan power | CFM / PJ in |
| Commercial Water Heating | thermal efficiency, SEF<sup>[7](#SEF)</sup>, COP | PJ out / PJ in |
| Residential Space Cooling | SEER<sup>[6](#AFUE)</sup>, EER, COP | PJ out / PJ in |
| Residential Lighting | system efficacy | Lumen-Hour/PJ |
| Residential Refrigeration | typical capacity over annual energy consumption | Cubic feet / PJ |
| Residential Space Heating | AFUE, thermal efficiency, HSPF<sup>[8](#HSPF)</sup>, COP, HHV<sup>[9](#HHV)</sup>| PJ out / PJ in |
| Residential Water Heating | UEF<sup>[7](#SEF)</sup>, SEF | PJ out / PJ in |

<a name="COP">1</a>: Coefficient of Performance (COP):  Energy efficiency rating measure determined, under specific testing conditions, by dividing the useful heating or cooling delivered by the required energy input.
<a name="EER">2</a>: Energy Efficiency Ratio (EER, IEER): A ratio representing the cooling capacity in Btu per hour by the power input in watts at any given set of rating conditions, expressed in Btu per hour per watt.
<a name="LIGHT">3</a>: Lighting efficiencies are represented by efficacy in lumens/watt.
<a name="TE">4</a>: Thermal efficiency (TE): the percentage of input heat energy that is transformed into useful work.
<a name="AFUE">5</a>: Annual Fuel Utilization Efficiency (AFUE): Efficiency rating based on average usage, including on and off cycling, as set out in the Department of Energy's standardized test procedures.
<a name="SEER">6</a>: Seasonal Energy Efficiency Ratio (SEER): The total cooling of a central unitary air conditioner or a unitary heat pump in Btu during its normal annual usage period for cooling divided by the total electric energy input in watt-hours during the same period. 
<a name="SEF">7</a>: Solar Energy Factor (SEF) and Uniform Energy Factor (UEF): defined as the energy delivered by the system divided the electrical or gas energy put into the system.
<a name="HSPF">8</a>: Heating Seasonal Performance Factor (HSPF): The total heating delivered by a heat pump in Btu during its normal annual usage period for heating divided by total electric input in watt-hours during the same period.
<a name="HHV">9</a>: Higher Heating Value (HHV): This thermal efficiency is fuel dependent and accounts for the latent heat of vaporization of water in the combustion products.

Several of the end-use service demands are defined by units of service rather than energy. In these cases, the specified efficiency has different input and output units, as noted below:

- Residential Lighting:
system efficacy – published in $\frac{Lumen}{W}$ and converted to $\frac{Lumen-h}{PJ}$ by dividing by $ 3600 \frac{J}{Wh} \times 10^{-15} \frac{PJ}{J}$ .

- Residential Refrigeration:
An efficiency is derived as the typical capacity over annual energy consumption where typical capacity is in units of ft<sup>3</sup> and annual energy consumption is in the units of kWh and subsequently converted to $\frac{ft^3}{PJ}$ by dividing by $ 3.6\times 10^{-9} \frac{PJ}{kWh} $.

- Commercial Ventilation:
An efficiency is derived as the system air flow over the system fan power, where system air flow is in units of ft<sup>3</sup>/min (CFM) and system fan power is in units of kW ($\frac{system air flow}{system fan power}$). The efficiency is thus in units of $\frac{CFM}{kW}$ and is subsequently converted to $\frac{CFM}{PJ/hour}$ by dividing by $10^{-12} \frac{PJ}{sec} \times 3600 \frac{sec}{hour} $.

- Commercial Refrigeration:
An efficiency is derived as the nominal capacity over average input, calculated as the cooling or heat rejection capacity over the annual energy consumption with units of $\frac{PJ in}{PJ out}$, where the cooling or heat rejection capacity is in units of $\frac{Btu}{hour}$ and the annual energy consumption is in units of $\frac{kWh}{Year}$. The unit conversions are as follows (Note that $\frac{PJ out}{PJ in} = \frac{Btu out}{But in}$ since the conversion factors cancel out):
\begin{equation*}
\frac{Capacity \frac{Btu}{hour} \times 24 \frac{hours}{day} \times 365 \frac{day}{year}}{Annual Energy Consumption \frac{kWh}{Year} \times 3421.14 \frac {Btu}{kWh}}
\end{equation*}.
- Commercial Lighting:
Similar to residential lighting, here conversion from electrical energy to lumens is represented by the system efficacy. The system efficacy is published in $\frac{Lumens}{Watt}$ and converted to $\frac{Lumens\cdot year}{PJ}$ by dividing by $ 3600 \frac{J}{Wh} \times 10^{-15} \frac{PJ}{J} \times 8760 \frac{hours}{year} $.

In [15]:
def show_efficiency(con):
    df_tech_desc = pd.read_sql("SELECT tech, tech_desc FROM technologies", con)
    df = pd.read_sql("SELECT regions, tech, vintage, efficiency FROM Efficiency WHERE tech IN (SELECT tech FROM technologies WHERE sector=='residential' OR sector='commercial')", con)
    df.loc[:,'agg_tech'] = df.loc[:,'tech'] #[map_plants[y] for x in df.tech for y in map_plants.keys() if y.lower() in x.lower()] #map agg technologies
    df_sum = df#df.drop("vintage", axis=1).groupby(by = ['regions','agg_tech']).sum().reset_index()
    df_sum['agg_tech'] = df_sum['agg_tech'].map(lambda x: df_tech_desc.loc[df_tech_desc.tech==x,'tech_desc'].values[0].replace('#','').strip())
    techs = ['All'] + list(df_sum.agg_tech.unique())
    regions = df_sum.regions.unique()
    def filter_tech(tech ='', region = ''):
        if tech=='All':
            df_sel = df_sum[(df_sum.regions==region)]
        else:
            df_sel = df_sum[(df_sum.agg_tech==tech) & (df_sum.regions==region)]
            df_sel.efficiency = 100*df_sel.efficiency
        df_sel = df_sel[['regions','tech','agg_tech','vintage','efficiency']]
        display(HTML(tabulate.tabulate(df_sel.set_index('regions'), ["regions", "technology", "description","vintage",'efficiency (%)' ], floatfmt=".1f", tablefmt='html')))

    w1 = widgets.Select(options=techs)
    w2 = widgets.Select(options=regions)
    w = widgets.interactive(filter_tech, tech=w1, region=w2)

    controls_rows(w)
    
show_efficiency(con)

### Existing Capacity <a class="anchor" id="exist_cap"></a>

To our knowledge, the existing installed capacities of the various technologies listed in the [EIA dataset](https://www.eia.gov/analysis/studies/buildings/equipcosts/) are not readily available. Sources like EFS and EIA, while critical in populating other parts of the database, do not report existing installed capacity. As a result, we lean on estimations from the USEPA9r MARKAL database [(Shay et al. 2013)](https://cfpub.epa.gov/si/si_public_record_Report.cfm?Lab=NRMRL&dirEntryID=150883) to fill this important gap. MARKAL spreadsheets report the existing capacities for most of the technologies listed in the EIA dataset for the [nine Census Divisions](https://www2.census.gov/geo/pdfs/maps-data/maps/reference/us_regdiv.pdf). However, since the OEO regions differ from the Census Divisions, we use the existing capacities reported in MARKAL and scale them using [U.S. Census state population data](https://www.census.gov/data/tables/time-series/demo/popest/2010s-state-total.html) to obtain estimates of existing capacity in each U.S. state. Finally, we aggregate up to the OEO regions and estimate the existing capacity of the EIA technologies. 

Within the MARKAL spreadsheet, the existing capacity is estimated by multiplying the demand met (taken from the [AEO](https://www.eia.gov/outlooks/aeo/)) in an end-use service subsector along with the estimated market share of a technology contributing to the end-use service category. This value is then divided by a utilization factor to estimate existing capacity of a certain technology in a given region. These data analysis steps are local to the MARKAL spreadsheet and we directly utilize the estimated existing capacities in the manner described above.

In [16]:
def show_exist_cap(con):
    df_tech_desc = pd.read_sql("SELECT tech, tech_desc FROM technologies", con)
    df = pd.read_sql("SELECT regions, tech,vintage, exist_cap, exist_cap_units FROM ExistingCapacity WHERE tech IN (SELECT tech FROM technologies WHERE sector=='residential' OR sector='commercial')", con)
    df.loc[:,'agg_tech'] = df.loc[:,'tech'] #[map_plants[y] for x in df.tech for y in map_plants.keys() if y.lower() in x.lower()] #map agg technologies
    df_sum = df.drop("vintage", axis=1).groupby(by = ['regions','tech','agg_tech','exist_cap_units']).sum().reset_index()
    df_sum.sort_values(by='exist_cap', ascending=False, inplace=True)
    df_sum['agg_tech'] = df_sum['agg_tech'].map(lambda x: df_tech_desc.loc[df_tech_desc.tech==x,'tech_desc'].values[0].replace('#','').strip())
    techs = ['All'] + list(df_sum.agg_tech.unique())
    regions = df_sum.regions.unique()
    def filter_tech(tech ='', region = ''):
        if tech=='All':
            df_sel = df_sum[(df_sum.regions==region)]
        else:
            df_sel = df_sum[(df_sum.agg_tech==tech) & (df_sum.regions==region)]
        display(HTML(tabulate.tabulate(df_sel.set_index('regions'), ["regions", "technology", "description", "units","capacity" ], floatfmt=".1f", tablefmt='html')))
    w1 = widgets.Select(options=techs)
    w2 = widgets.Select(options=regions)
    w = widgets.interactive(filter_tech, tech=w1, region=w2)

    controls_rows(w)
    
show_exist_cap(con)


### Discount Rate <a class="anchor" id="disc_rates"></a>
All new capacity is given a technology-specific discount rate of 30%. Without this specification, Temoa's least cost optimization results in the early replacement of virtually all existing demand technologies with new capacity at higher efficiency. Thus this 30% rate is used to ensure that existing capacity is utilized for the remainder of its lifetime under base case assumptions. In the future, we plan to update this hurdle rate based on calibrations from the updated OEO database.

In [17]:
def show_disc_rate(con):
    df_tech_desc = pd.read_sql("SELECT tech, tech_desc FROM technologies", con)
    df = pd.read_sql("SELECT regions, tech, vintage, tech_rate FROM DiscountRate \
    WHERE tech in (SELECT tech FROM technologies WHERE sector='residential' OR sector='commercial')", con)
    df['agg_tech'] = df['tech'].map(lambda x: df_tech_desc.loc[df_tech_desc.tech==x,'tech_desc'].values[0].replace('#','').strip())
    techs = []
    for unique_tech in df.agg_tech.unique():
        techs.append(unique_tech)
    techs = ['All'] + list(set(techs))
    regions = df.regions.unique()
    def filter_tech(tech='', region =''):
        if tech=='All':
            df_sel = df[(df.regions==region)]
        else:
            df_sel = df[(df.agg_tech==tech) & (df.regions==region)]
        df_sel = df_sel.pivot_table(index=['regions','tech', 'agg_tech'], columns='vintage', values='tech_rate').reset_index()
        df_sel.rename(columns={'agg_tech':'description'},inplace=True)
        header = list(df_sel.columns.values)
        display(HTML(tabulate.tabulate(df_sel.set_index('regions'), header, floatfmt=".3f", tablefmt='html')))
    w1 = widgets.Select(options=techs)
    w2 = widgets.Select(options=regions)
    w = widgets.interactive(filter_tech, tech=w1, region=w2)

    controls_rows(w)
    
show_disc_rate(con)

## Look-up Tools <a class="anchor" id="look_up_tools"></a>

### Technology/Commodity Description Look-up Tool <a class="anchor" id="description_look_up"></a>
Use the tool below to search for any key words that may describe a technology or commodity of interest (e.g. heating, cooling). The tool provides a list of all the technologies in the database that may be relevant to the query.

In [18]:
w = widgets.Text(value='space heating demand')
display(w)
def f(w):
    if len(w)>0:
        df1 = pd.read_sql("SELECT * FROM commodities WHERE comm_desc LIKE '%" + w + "%'", con)
        df1['desc'] = df1['comm_desc'].str.replace('#','').str.strip()
        df1['comm_tech'] = df1['comm_name']
        df1['type'] = 'commodity'

        df2 = pd.read_sql("SELECT * FROM technologies WHERE tech_desc LIKE '%" + w + "%'", con)
        df2['desc'] = df2['tech_desc'].str.replace('#','').str.strip()
        df2['comm_tech'] = df2['tech']
        df2['type'] = 'technology'


        df = pd.concat([df1[['comm_tech','type','desc']], df2[['comm_tech','type','desc']]])
        
        if len(df)>0:
            display(HTML(tabulate.tabulate(df.set_index('comm_tech'),['technology/commodity','type','description'],tablefmt='html')))
        else:
            print('')
    else:
        print('')
            


out = widgets.interactive_output(f, {'w': w})
display(out)

Text(value='space heating demand')

Output()

### Network Diagram Look-up Tool  <a class="anchor" id="network_look_up"></a>
Use the [description lookup tool](#description_look_up) above to identify specific commodity or technology names. Type the name in the box below to generate a corresponding network diagram for that commodity or technology. The slider can be used to view different upstream levels of the network diagram.

In [19]:
w = widgets.Text(value='RSH')
display(w)
def f(w):
    if len(w)>0:
        
        df1 = pd.read_sql("SELECT comm_name, comm_desc FROM commodities WHERE comm_name='" + w + "'", con)
        df1['desc'] = df1['comm_desc'].str.replace('#','').str.strip()
        df1['comm_tech'] = df1['comm_name']
        df1['type'] = 'commodity'

        df2 = pd.read_sql("SELECT * FROM technologies WHERE tech='" + w + "'", con)
        df2['desc'] = df2['tech_desc'].str.replace('#','').str.strip()
        df2['comm_tech'] = df2['tech']
        df2['type'] = 'technology'

        df = pd.concat([df1[['comm_tech','type','desc']], df2[['comm_tech','type','desc']]])

    
        if len(df)>0:
            def show_desc(level):
                display(Markdown(df['desc'][0]))
                final_dem = df['comm_tech'][0]
                df_graph = return_flowd_table(final_dem,level)
                args = create_args_flowd(df_graph)
                colors, quick_run_dot_fmt = return_format_colors()
                args.update(colors)
                #o_str = 'rankdir = "LR" ;'
                #r_str = 'rankdir = "LR" ; \n\t size="12,12";'
                #quick_run_dot_fmt = quick_run_dot_fmt.replace(o_str, r_str)
                dot_graph = quick_run_dot_fmt % args
                display(graphviz.Source(dot_graph))
            w2 = widgets.IntSlider(value=1,min=0,max=10,step=1,description='Level:',disabled=False,continuous_update=True,orientation='horizontal',readout=True,readout_format='d')
            w = widgets.interactive(show_desc, level=w2)
            controls_rows(w)
        else:
            print('')

    else:
        print('')
            


out = widgets.interactive_output(f, {'w': w})
display(out)

Text(value='RSH')

Output()

###  Technology/Commodity Look-up Tool <a class="anchor" id="tech_lookup"></a>
Use the tool below to retrieve the description for any technology or commodity within the database. Type the commodity or technology name in the box below to view the description. Note that names are case sensitive.

In [20]:
w = widgets.Text(value='RSH')
display(w)
def f(w):
    df = pd.read_sql("SELECT * FROM commodities WHERE comm_name='" + w + "'", con)
    if len(df)==0:
        df = pd.read_sql("SELECT * FROM technologies WHERE tech='" + w + "'", con)

    if len(df)>0:
        try:
            display(Markdown((df['comm_desc'].values[0].replace('#', '').strip())))
        except:
            display(Markdown(df['tech_desc'].values[0].replace('#', '').strip()))
    else:
        print('')


out = widgets.interactive_output(f, {'w': w})
display(out)


Text(value='RSH')

Output()